# Chapter 6: Data Loading, Storage, and File Formats

- Đọc dữ liệu và làm cho dữ liệu có thể truy cập (data loading) là bước đầu tiên cần thiết.
- Thuật ngữ parsing cũng dùng để chỉ việc nạp dữ liệu văn bản và chuyển thành bảng với các kiểu dữ liệu khác nhau.
- Tập trung vào input/output bằng pandas, mặc dù còn nhiều thư viện khác hỗ trợ các định dạng dữ liệu khác nhau.
- Input/output thường gồm: đọc file văn bản hoặc các định dạng lưu trữ hiệu quả, tải dữ liệu từ cơ sở dữ liệu, và tương tác với nguồn mạng như web API.

## 6.1 Reading and Writing Data in Text Format

In [123]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)

> Hãy bắt đầu với một tệp văn bản CSV (comma-separated values) nhỏ:

In [124]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

- Vì file ngăn cách bằng dấu phẩy, có thể dùng `pandas.read_csv` để đọc vào DataFrame.

In [125]:
df = pd.read_csv("examples/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- Một tệp không phải lúc nào cũng có hàng tiêu đề. Hãy xem xét tệp sau:

In [126]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

- Để đọc tệp này, bạn có một vài lựa chọn. Bạn có thể để pandas tự gán tên cột mặc định, hoặc bạn có thể tự chỉ định tên cột:

In [127]:
pd.read_csv("examples/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [128]:
pd.read_csv("examples/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- Giả sử bạn muốn cột message trở thành chỉ mục (index) của DataFrame trả về. Bạn có thể chỉ định cột ở vị trí số 4 hoặc cột có tên "message" bằng cách sử dụng tham số index_col:

In [129]:
names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


- Nếu bạn muốn tạo một chỉ mục phân cấp (hierarchical index) (sẽ được thảo luận trong Mục 8.1, “Đánh chỉ mục phân cấp,” ở trang 247) từ nhiều cột, hãy truyền vào một danh sách các số thứ tự cột hoặc tên cột:

In [130]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [131]:
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

- Trong một số trường hợp, bảng không có delimiter cố định, dùng dấu cách hoặc mẫu khác để phân tách các trường.

> Xem ví dụ file văn bản sau.

In [132]:
!cat examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


- Mặc dù có thể xử lý thủ công, nhưng các trường ở đây được phân tách bằng lượng whitespace biến đổi.
- Trong trường hợp này, có thể truyền regular expression làm delimiter cho `pandas.read_csv`.

> Ví dụ dùng regex \s+:

In [133]:
result = pd.read_csv("examples/ex3.txt", sep=r"\s+")
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


- Vì số tên cột ít hơn số hàng dữ liệu, pandas đọc file suy đoán cột đầu tiên là index trong trường hợp đặc biệt này.
- Các hàm parsing có nhiều tham số bổ sung để xử lý các định dạng file ngoại lệ.

> Ví dụ: bỏ qua hàng 1, 3 và 4 bằng skiprows.

In [134]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [135]:
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- Xử lý dữ liệu thiếu là phần quan trọng và thường tinh vi khi đọc file.
- Dữ liệu thiếu thường là không có giá trị (chuỗi rỗng) hoặc được đánh dấu bằng sentinel (giá trị đại diện).
- Mặc định, pandas dùng một tập các sentinel phổ biến như NA và NULL.

In [136]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [137]:
result = pd.read_csv("examples/ex5.csv")

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


- Nhớ rằng pandas hiển thị giá trị thiếu dưới dạng NaN, nên trong kết quả có hai giá trị null/missing.

In [138]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


- Tham số `na_values` nhận một dãy chuỗi để thêm vào danh sách mặc định các giá trị được pandas coi là missing.

In [139]:
result = pd.read_csv("examples/ex5.csv", na_values=["NULL"])

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


- `pandas.read_csv` có danh sách nhiều giá trị NA mặc định, nhưng có thể vô hiệu hóa bằng `keep_default_na=False`.

In [140]:
result2 = pd.read_csv("examples/ex5.csv", keep_default_na=False)

result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [141]:

result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [142]:
result3 = pd.read_csv("examples/ex5.csv", keep_default_na=False, na_values=["NA"])
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [143]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


- Có thể chỉ định các giá trị NA khác nhau cho từng cột bằng một dictionary.

In [144]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("examples/ex5.csv", na_values=sentinels, keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


### Reading Text Files in Pieces

- Khi xử lý file rất lớn hoặc tìm tập tham số phù hợp, bạn có thể đọc một phần nhỏ hoặc lặp qua các chunk nhỏ của file.
- Trước khi xem file lớn, điều chỉnh cài đặt hiển thị của pandas cho gọn hơn.

In [145]:
pd.options.display.max_rows = 10

- giờ bạn có thể:

In [146]:
result = pd.read_csv("examples/ex6.csv")

result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


- Dấu `...` cho biết các hàng ở giữa DataFrame bị bỏ qua.
- Nếu muốn chỉ đọc một số hàng nhỏ (không đọc toàn bộ file), dùng tham số nrows.

In [147]:
pd.read_csv("examples/ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


- Để đọc file theo từng phần, dùng `chunksize` để chỉ định số lượng hàng mỗi lần đọc.

In [148]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
type(chunker)

pandas.io.parsers.readers.TextFileReader

- `pandas.read_csv` trả về TextFileReader, cho phép lặp qua các phần của file theo chunksize.

> Ví dụ: có thể lặp qua ex6.csv, tính tổng số lần xuất hiện của các giá trị trong cột "key".

In [149]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)

- Kết quả thu được như sau:

In [150]:
tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing Data to Text Format

- Dữ liệu cũng có thể được xuất ra định dạng phân tách. 

> Ví dụ, với một file CSV đã đọc trước đó:

In [151]:
data = pd.read_csv("examples/ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


- Dùng phương thức `to_csv` của DataFrame để ghi dữ liệu ra file CSV (phân tách bằng dấu phẩy).

In [152]:
data.to_csv("examples/out.csv")
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


- Có thể sử dụng các ký tự phân tách khác, ví dụ ghi ra `sys.stdout` để hiển thị kết quả trên console thay vì file.

In [153]:
import sys
data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


- Các giá trị thiếu (missing values) sẽ xuất hiện dưới dạng chuỗi rỗng khi xuất ra.
- Có thể thay thế bằng một giá trị đặc biệt khác để biểu thị missing data

In [154]:
data.to_csv(sys.stdout, na_rep="NULL")


,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


- Nếu không chỉ định thêm tùy chọn nào, cả nhãn hàng và cột đều được ghi ra.
- Có thể tắt việc ghi nhãn hàng hoặc cột nếu muốn.

In [155]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


- Có thể ghi ra chỉ một số cột nhất định và theo thứ tự tùy chọn.

In [156]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


### Working with Other Delimited Formats

- Hầu hết dữ liệu dạng bảng có thể được nạp từ đĩa bằng các hàm như pandas.read_csv.
- Tuy nhiên, đôi khi cần xử lý thủ công nếu file có các dòng sai định dạng, vì chúng có thể gây lỗi khi đọc bằng pandas.

In [157]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


- Với các file có dấu phân cách một ký tự, bạn có thể dùng module csv tích hợp sẵn của Python.
- Sử dụng csv.reader và truyền vào file đã mở hoặc đối tượng giống file.

In [158]:
import csv
f = open("examples/ex7.csv")
reader = csv.reader(f)

- Khi lặp qua đối tượng reader giống như file, bạn nhận được danh sách các giá trị của mỗi dòng.
- Các ký tự ngoặc kép (quotes) sẽ được loại bỏ tự động.

In [159]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [160]:
f.close()

- Bước tiếp theo là xử lý dữ liệu thủ công để đưa vào dạng mong muốn.
- Trước tiên, đọc file thành một danh sách các dòng (list of lines).

In [161]:
with open("examples/ex7.csv") as f: 
    lines = list(csv.reader(f))

- Sau đó, tách các dòng thành dòng tiêu đề (header line) và các dòng dữ liệu (data lines).

In [162]:
header, values = lines[0], lines[1:]

- Tiếp theo, tạo một dictionary các cột dữ liệu bằng dictionary comprehension và biểu thức `zip(*values)`, chuyển đổi hàng thành cột. (Lưu ý: có thể tốn nhiều bộ nhớ với file lớn)

In [163]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

- File CSV có nhiều dạng khác nhau.
- Để định nghĩa một định dạng mới với dấu phân tách khác, cách trích dẫn chuỗi khác, hoặc kết thúc dòng khác, ta có thể tạo một subclass của `csv.Dialect`.

### JSON Data

- JSON (JavaScript Object Notation) là định dạng tiêu chuẩn để gửi dữ liệu qua HTTP giữa trình duyệt web và các ứng dụng khác.
- JSON tự do hơn CSV, không bị giới hạn theo dạng bảng.

> Ví dụ minh họa một cấu trúc JSON:

In [164]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

- JSON gần như là mã Python hợp lệ, chỉ khác một số điểm như giá trị `null` và không cho phép dấu phẩy cuối trong danh sách.

Các kiểu cơ bản trong JSON: object (dict), array (list), string, number, Boolean, null.

Tất cả các key trong object phải là string.

Python có nhiều thư viện để đọc/ghi JSON; ở đây dùng thư viện chuẩn `json`.

Chuyển JSON string → Python object dùng `json.loads()`.

In [165]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

- Ngược lại, `json.dumps` chuyển một Python object → JSON string.

In [166]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

- Bạn có thể chuyển một JSON object (hoặc danh sách JSON) thành DataFrame bằng cách truyền danh sách từ điển (tương ứng với các JSON object) vào DataFrame, đồng thời chọn một số trường dữ liệu cần thiết

In [167]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

,name,age
0,Scott,34
1,Katie,42


- Hàm pandas.read_json có thể tự động chuyển các tập dữ liệu JSON được sắp xếp theo định dạng cụ thể thành Series hoặc DataFrame.

In [168]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


- Các tùy chọn mặc định của `pandas.read_json` giả định rằng mỗi đối tượng trong mảng JSON tương ứng với một hàng trong bảng.

In [169]:
data = pd.read_json("examples/example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


- Có thể xuất dữ liệu từ pandas sang JSON bằng cách dùng phương thức to_json của Series hoặc DataFrame.

In [170]:
data.to_json(sys.stdout)
data.to_json(sys.stdout, orient="records")

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

### XML and HTML: Web Scraping

- Python có nhiều thư viện để đọc và ghi dữ liệu từ HTML và XML, như lxml, Beautiful Soup, và html5lib. Trong đó, lxml nhanh hơn, còn các thư viện khác xử lý HTML/XML bị lỗi tốt hơn.
- Pandas cung cấp pandas.read_html để tự động trích xuất bảng từ HTML thành DataFrame. Kết quả là một danh sách các DataFrame, mặc định đọc tất cả các bảng trong thẻ <table>.

In [171]:
pip install lxml



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [172]:
tables = pd.read_html("examples/fdic_failed_bank_list.html")
len(tables)
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


- Vì failures có nhiều cột, pandas chèn ký tự xuống dòng `\`.
- Sau đó, dữ liệu có thể được làm sạch và phân tích.

> Ví dụ: tính số lần ngân hàng thất bại theo từng năm.

In [173]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

Closing Date
2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: count, Length: 15, dtype: int64

### Parsing XML with lxml.objectify

- Sử dụng lxml.objectify để phân tích (parse) file XML.
- Dùng getroot để lấy tham chiếu đến nút gốc (root node) của file XML.

In [174]:
from lxml import objectify

path = "datasets/mta_perf/Performance_MNR.xml"
with open(path) as f:
    parsed = objectify.parse(f)
root = parsed.getroot()

- `root.INDICATOR` trả về generator, sinh ra từng phần tử `<INDICATOR>` trong XML.
- Với mỗi bản ghi, có thể tạo dictionary ánh xạ tên thẻ (tag) như YTD_ACTUAL tới giá trị dữ liệu, bỏ qua một số thẻ nhất định.

In [175]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

- Cuối cùng, chuyển danh sách các dictionary thành DataFrame để dễ dàng xử lý và phân tích dữ liệu.

In [176]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


- `pandas.read_xml` giúp chuyển XML thành DataFrame nhanh chóng, thay thế cho việc xử lý từng thẻ XML thủ công.

In [177]:
perf2 = pd.read_xml(path)
perf2.head()

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80


## 6.2 Binary Data Formats

- Một cách đơn giản để lưu dữ liệu ở định dạng nhị phân là dùng module pickle của Python.
- Các đối tượng của pandas có phương thức `to_pickle` để ghi dữ liệu ra đĩa dưới định dạng pickle.

In [178]:
frame = pd.read_csv("examples/ex1.csv")
frame
frame.to_pickle("examples/frame_pickle")

- File pickle chỉ đọc được trong Python.
- Để đọc các đối tượng đã lưu dưới dạng pickle, có thể dùng module pickle hoặc thuận tiện hơn là `pandas.read_pickle`.

In [179]:
pd.read_pickle("examples/frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- pandas hỗ trợ một số định dạng dữ liệu nhị phân mở như HDF5, ORC, Parquet.

> Ví dụ, sau khi cài pyarrow (conda install pyarrow), có thể đọc file Parquet bằng pandas.read_parquet.

In [180]:
!rm examples/frame_pickle

In [181]:
!pip uninstall pyarrow -y
!pip install pyarrow --upgrade
!pip install fastparquet --upgrade



Found existing installation: pyarrow 21.0.0
Uninstalling pyarrow-21.0.0:
  Successfully uninstalled pyarrow-21.0.0
  Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (42.8 MB)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [182]:
# fec = pd.read_parquet('datasets/fec/fec.parquet')

fec = pd.read_parquet('datasets/fec/fec.parquet', engine='fastparquet')

### Reading Microsoft Excel Files

- Để dùng `pandas.ExcelFile`, tạo một instance bằng cách truyền đường dẫn đến file .xls hoặc .xlsx.

In [183]:
!pip install openpyxl


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [184]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")

- Đối tượng `ExcelFile` có thể hiển thị danh sách các tên sheet có trong file.

In [185]:
xlsx.sheet_names

['Sheet1']

- Dữ liệu trong một sheet có thể được đọc vào DataFrame bằng phương thức `parse`.

In [186]:
xlsx.parse(sheet_name="Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


- Nếu bảng Excel có cột chỉ mục, có thể chỉ định bằng đối số `index_col` khi dùng `parse`.

In [187]:
xlsx.parse(sheet_name="Sheet1", index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- Đọc nhiều sheet trong cùng file: dùng `pandas.ExcelFile` nhanh hơn, nhưng vẫn có thể đọc trực tiếp bằng `pandas.read_excel`.

In [188]:
frame = pd.read_excel("examples/ex1.xlsx", sheet_name="Sheet1")
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


- Để ghi dữ liệu pandas ra file Excel: tạo `ExcelWriter`, sau đó dùng `to_excel` để xuất dữ liệu.

In [189]:
import sys
!{sys.executable} -m pip install openpyxl


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [190]:

writer = pd.ExcelWriter("examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.close()

/tmp/ipykernel_5496/1674296039.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  frame.to_excel(writer, "Sheet1")


- Có thể trực tiếp truyền file path vào to_excel mà không cần dùng ExcelWriter.

In [191]:
frame.to_excel("examples/ex2.xlsx")

### Using HDF5 Format

- pandas cung cấp HDFStore, một giao diện cấp cao để lưu trữ Series và DataFrame dưới định dạng HDF5, hoạt động giống như dictionary và xử lý các chi tiết cấp thấp.

In [192]:
!rm examples/ex2.xlsx

In [193]:
!rm -f examples/mydata.h5

In [195]:
!pip install tables

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.2/546.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [tables]2m5/6 [tables]]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [196]:
frame = pd.DataFrame({"a": np.random.standard_normal(100)})
store = pd.HDFStore("examples/mydata.h5")
store["obj1"] = frame
store["obj1_col"] = frame["a"]
store

<class 'pandas.io.pytables.HDFStore'>
File path: examples/mydata.h5

- Các đối tượng trong file HDF5 có thể được truy xuất bằng cùng một API giống dictionary.

In [197]:
store["obj1"]

,a
0,-1.565657
1,-0.562540
2,-0.032664
3,-0.929006
4,-0.482573
...,...
95,0.910983
96,-1.020903
97,-1.413416
98,1.296608


- HDFStore hỗ trợ hai kiểu lưu trữ: `"fixed"` và `"table"` (mặc định là `"fixed"`).
- `"table"` chậm hơn nhưng hỗ trợ truy vấn bằng cú pháp đặc biệt.

In [198]:
store.put("obj2", frame, format="table")
store.select("obj2", where=["index >= 10 and index <= 15"])
store.close()

- `put` là phiên bản rõ ràng của `store["obj2"] = frame`, cho phép cài đặt thêm các tùy chọn như định dạng lưu trữ.
- `pandas.read_hdf` cung cấp một phím tắt để đọc dữ liệu từ HDF5.

In [199]:
frame.to_hdf("examples/mydata.h5", "obj3", format="table")
pd.read_hdf("examples/mydata.h5", "obj3", where=["index < 5"])

/tmp/ipykernel_5496/2659235977.py:1: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  frame.to_hdf("examples/mydata.h5", "obj3", format="table")


,a
0,-1.565657
1,-0.562540
2,-0.032664
3,-0.929006
4,-0.482573


> Bạn có thể xóa file HDF5 vừa tạo bằng lệnh:

In [200]:
import os
os.remove("examples/mydata.h5")

## 6.3 Interacting with Web APIs

- Có thể lấy 30 issue gần nhất của pandas trên GitHub bằng cách gửi một HTTP GET request sử dụng thư viện requests.

In [201]:
import requests
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.raise_for_status()
resp

<Response [200]>

- Luôn gọi `raise_for_status` sau `requests.get` để kiểm tra lỗi HTTP.
- Dùng `response.json()` để nhận dữ liệu JSON đã được phân tích cú pháp thành dictionary hoặc list trong Python.

In [202]:
data = resp.json()
data[0]["title"]

'DEPR: slicing with a date object'

- Kết quả lấy được dựa trên dữ liệu thời gian thực, nên khi chạy code sẽ khác so với ví dụ.
- Mỗi phần tử trong `data` là một dictionary chứa tất cả thông tin trên trang issue của GitHub (trừ phần bình luận).
- Có thể truyền `data` trực tiếp vào `pandas.DataFrame` và trích xuất các trường dữ liệu quan tâm.

In [203]:
issues = pd.DataFrame(data, columns=["number", "title",
                                     "labels", "state"])
issues

,number,title,labels,state
0,62428,DEPR: slicing with a date object,[],open
1,62427,DOC: Validate docstrings for pandas.Period.freq,[],open
2,62425,[WIP] DOC: Add guide links to relevant API docs,[],open
3,62424,BUG: String[pyarrow] comparison with mixed object,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.g...",open
4,62423,"API/REF: consistent error messages, arithmetic rules","[{'id': 47223669, 'node_id': 'MDU6TGFiZWw0NzIyMzY2OQ==', 'url': 'https:...",open
...,...,...,...,...
25,62358,DOC: Expand data table respresentation,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OTk=', 'url': 'https://api....",open
26,62357,DOC: Ensure guides are linked from relevant API docs,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OTk=', 'url': 'https://api....",open
27,62356,BUG: Merge fails on pyarrow datetime columns,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.g...",open
28,62353,ENH: arithmetic between DatetimeArray / TimedeltaArray and list,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg==', 'url': 'https://api.g...",open


## 6.4 Interacting with Databases

- Trong môi trường doanh nghiệp, nhiều dữ liệu không được lưu dưới dạng file văn bản hay Excel.
- Các cơ sở dữ liệu quan hệ dựa trên SQL (SQL Server, PostgreSQL, MySQL) rất phổ biến, cùng với một số cơ sở dữ liệu thay thế khác.
- Lựa chọn cơ sở dữ liệu phụ thuộc vào hiệu năng, tính toàn vẹn dữ liệu và khả năng mở rộng của ứng dụng.
- pandas cung cấp các hàm giúp dễ dàng tải kết quả của truy vấn SQL vào DataFrame.

> Ví dụ, có thể tạo cơ sở dữ liệu SQLite3 bằng Python với driver sqlite3 tích hợp sẵn.

In [204]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect("mydata.sqlite")
con.execute(query)
con.commit()

- Sau khi tạo cơ sở dữ liệu, chèn một vài hàng dữ liệu vào bảng đã tạo.

In [205]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

- Hầu hết các trình điều khiển SQL trong Python trả về dữ liệu dưới dạng một danh sách các tuple khi thực hiện truy vấn SELECT từ bảng.

In [206]:
cursor = con.execute("SELECT * FROM test")
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

- Có thể truyền danh sách các tuple vào constructor của DataFrame.
- Cần cung cấp tên cột, có trong thuộc tính description của cursor.
- Với SQLite3, cursor.description chỉ cung cấp tên cột; các trường khác là None.
- Một số driver cơ sở dữ liệu khác cung cấp thêm thông tin về cột.

In [207]:

cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


- Kết nối đến cùng cơ sở dữ liệu SQLite bằng SQLAlchemy.
- Đọc dữ liệu từ bảng đã tạo trước đó.

In [209]:
!pip install SQLAlchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [SQLAlchemy]2 [SQLAlchemy]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [210]:
import sqlalchemy as sqla
db = sqla.create_engine("sqlite:///mydata.sqlite")
pd.read_sql("SELECT * FROM test", db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [211]:
!rm mydata.sqlite

## 6.5 Conclusion

- Truy cập dữ liệu thường là bước đầu tiên trong quá trình phân tích dữ liệu.
- Chương này đã giới thiệu một số công cụ hữu ích để bắt đầu.
- Các chương tiếp theo sẽ đi sâu hơn vào xử lý dữ liệu, trực quan hóa dữ liệu, phân tích chuỗi thời gian và các chủ đề khác.